In [2]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import requests


In [3]:

# Load the dataset from CSV file
df = pd.read_csv('weather_mood_data.csv')
df.tail()


,temperature,cloud_pct,humidity,wind_speed,mood
695,28.292230,89.934391,40.487253,15.359515,sad
696,32.796185,58.990561,48.279582,18.266209,sad
697,27.170618,44.542504,29.682588,15.261211,happy
698,15.775524,19.570967,43.664527,3.725218,sad
699,30.174001,32.220352,21.401220,18.273886,sad


In [4]:

# Split the data into features (X) and target variable (y)
X = df.drop('mood', axis=1)
y = df['mood']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Train the classifier
clf.fit(X_train, y_train)


DecisionTreeClassifier()

In [10]:

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 71.43%


In [11]:

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

       happy       0.55      0.64      0.59        45
         sad       0.82      0.75      0.78        95

    accuracy                           0.71       140
   macro avg       0.68      0.70      0.69       140
weighted avg       0.73      0.71      0.72       140



In [12]:

# use the trained model to predict mood based on new weather data
new_data = {'temperature': [22], 'cloud_pct': [40], 'humidity': [55], 'wind_speed': [8]}
new_df = pd.DataFrame(new_data)

# Make predictions on new data
new_predictions = clf.predict(new_df)
print(f"Predicted mood for new data: {new_predictions[0]}")


Predicted mood for new data: happy


In [13]:
song_df = pd.read_csv('song.csv')
song_df.head()

,id,Valence,Artist,Track,Album,Album_type,Danceability,Energy,Loudness,Speechiness,...,Title,Channel,Views,Likes,Comments,Licensed,official_video,Stream,EnergyLiveness,most_playedon
0,1,0.772,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,0.705,-6.679,0.1770,...,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221,6220896,169907,True,True,1040234854,1.150082,Spotify
1,2,0.852,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,0.703,-5.815,0.0302,...,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645,1079128,31003,True,True,310083733,15.183585,Spotify
2,3,0.551,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,0.923,-3.930,0.0522,...,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055,282142,7399,True,True,63063467,7.956897,Spotify
3,4,0.578,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,0.739,-5.810,0.0260,...,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952,1788577,55229,True,True,434663559,11.546875,Spotify
4,5,0.525,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,0.694,-8.627,0.1710,...,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958,6197318,155930,True,True,617259738,9.942693,Youtube


Weather API call

In [14]:
def get_weather_data(city):
    url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"
    querystring = {"city": city}

    headers = {
        "X-RapidAPI-Key": "291c38cd95msh3f1c2c176666ab0p1c53b6jsn717111b85e3f",
        "X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com",
    }

    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

Detect Emotion from weather using decision tree

In [15]:
def detect_weather_mood(city):
    weather_data = get_weather_data(city)
    
    # Extract relevant weather information
    temperature = weather_data['temp']
    humidity = weather_data['humidity']
    cloud_pct = weather_data['cloud_pct']
    wind_speed = weather_data['wind_speed']

    # Create a DataFrame for the new data
    new_data = pd.DataFrame({
        'temperature': [temperature],
        'cloud_pct': [cloud_pct],
        'humidity': [humidity],
        'wind_speed': [wind_speed]
    })
    
    new_prediction = clf.predict(new_data)[0]


    return new_prediction


In [16]:
detect_weather_mood('Khulna')

'sad'

### Song Dataset 

In [17]:
col_to_keep = ['id', 'Track', 'Valence', 'Views']

col_to_drop = [col for col in song_df.columns if col not in col_to_keep]

song_df = song_df.drop(columns=col_to_drop)

## drop rows with null values
song_df.dropna()

song_df = song_df.sort_values(by='Views', ascending=False)

print(song_df.shape)

song_df.head()

(18336, 4)


,id,Valence,Track,Views
360,361,0.839,Despacito,8079646911
11535,11536,0.931,Shape of You,5908398479
11551,11552,0.283,See You Again (feat. Charlie Puth),5773797147
18018,18019,0.965,Wheels on the Bus,4898831101
9952,9953,0.928,Uptown Funk (feat. Bruno Mars),4821016218


### song recommendation from city weather

In [18]:
def weather_to_songs_suggest():
    cityName = input("City name: ")

    # Use the decision tree model to predict mood
    mood = detect_weather_mood(cityName)

    recommendations = []

    for index, row in song_df.iterrows():
        if mood == 'happy' and row['Valence'] >= 0.7:
            recommendations.append(row)
        elif mood == 'sad' and row['Valence'] <= 0.3:
            recommendations.append(row)
        
        if len(recommendations) == 5000:
            break
    
    recommendations.sort(key=lambda x: x['Views'], reverse=True)
    
    for i, song in enumerate(recommendations[:20]):
        print(i + 1, '.', song['Track'])


In [19]:
weather_to_songs_suggest()

1 . See You Again (feat. Charlie Puth)
2 . Faded
3 . Let Her Go - Acoustic
4 . Let Her Go
5 . Perfect
6 . Lean On
7 . Mi Gente
8 . Te Boté
9 . A Thousand Years
10 . November Rain
11 . Someone Like You
12 . Dusk Till Dawn (feat. Sia) - Radio Edit
13 . Thunder
14 . The Hills
15 . Numb
16 . lovely (with Khalid)
17 . Paradise
18 . Bohemian Rhapsody - Remastered 2011
19 . Radioactive
20 . Zaroori Tha
